#### Annotated Types

* `NamedTuple`

In [1]:
from typing import NamedTuple
from typing_extensions import TypedDict
from pydantic import BaseModel, Extra, ValidationError

In [2]:
class Point(NamedTuple):
    x: int
    y: int

In [3]:
class PointModel(BaseModel):
    p: Point

In [4]:
print(f"{PointModel(p=('1', '2')) = }")

PointModel(p=('1', '2')) = PointModel(p=Point(x=1, y=2))


In [5]:
try:
    m = PointModel(p=('1.3', '2'))
    print(m)
except ValidationError as e:
    print(e)

1 validation error for PointModel
p -> x
  value is not a valid integer (type=type_error.integer)


* `TypedDict`

> ##### Note
>
> This is a new feature of the Python standard library as of Python 3.8. Prior to Python 3.8, it requires the `typing-extensions` package. But required and optional fields are properly differentiated only since Python 3.9. We therefore recommend using `typing-extensions` with Python 3.8 as well.

In [6]:
# `total=False` means keys are non-required
class UserIdentity(TypedDict, total=False):
    name: str
    surname: str

In [7]:
class User(TypedDict):
    identity: UserIdentity
    age: int

In [8]:
class UserModel(BaseModel):
    u: User
    
    class Config:
        extra = Extra.forbid

In [9]:
print(f"{UserModel(u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': '37'}) = }")

UserModel(u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': '37'}) = UserModel(u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': 37})


In [10]:
print(f"{UserModel(u={'identity': {'name': None, 'surname': 'John'}, 'age': '37'}) = }")

UserModel(u={'identity': {'name': None, 'surname': 'John'}, 'age': '37'}) = UserModel(u={'identity': {'name': None, 'surname': 'John'}, 'age': 37})


In [11]:
print(f"{UserModel(u={'identity': {}, 'age': '37'}) = }")

UserModel(u={'identity': {}, 'age': '37'}) = UserModel(u={'identity': {}, 'age': 37})


In [12]:
try:
    m = UserModel(u={"identity": {"name": ["Smith"], "surname": "John"}, "age": "24"})
    print(m)
except ValidationError as e:
    print(e)

1 validation error for UserModel
u -> identity -> name
  str type expected (type=type_error.str)


In [13]:
try:
    m = UserModel(
        u={
            "identity": {"name": "Smith", "surname": "John"},
            "age": "37",
            "email": "john.smith@me.com",
        }
    )
except ValidationError as e:
    print(e)

1 validation error for UserModel
u -> email
  extra fields not permitted (type=value_error.extra)
